In [1]:
%matplotlib inline
import sys
import copy
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt

import pcse
from pcse.models import Wofost72_WLP_FD
print("This notebook was built with:")
print(f"python version: {sys.version}")
print(f"PCSE version: {pcse.__version__}")

from dataproviders import parameters, agromanagement, weather
wofost = Wofost72_WLP_FD(parameters, weather, agromanagement)
wofost.run_till_terminate()
df = pd.DataFrame(wofost.get_output()).set_index("day")

This notebook was built with:
python version: 3.9.19 (main, Apr  6 2024, 17:57:55) 
[GCC 9.4.0]
PCSE version: 5.5.6


In [2]:
import pypfilt

class Lorenz63(pypfilt.OdeModel):
    def field_types(self, ctx):
        r"""
        Define the state vector :math:`[\sigma, \rho, \beta, x, y, z]^T`.
        """
        return [
            ('sigma', float),
            ('rho', float),
            ('beta', float),
            ('x', float),
            ('y', float),
            ('z', float),
        ]

    def d_dt(self, time, xt, ctx, is_forecast):
        """
        The right-hand side of the ODE system.

        :param time: The current time.
        :param xt: The particle state vectors.
        :param ctx: The simulation context.
        :param is_forecast: True if this is a forecasting simulation.
        """
        rates = np.zeros(xt.shape, xt.dtype)
        rates['x'] = xt['sigma'] * (xt['y'] - xt['x'])
        rates['y'] = xt['x'] * (xt['rho'] - xt['z']) - xt['y']
        rates['z'] = xt['x'] * xt['y'] - xt['beta'] * xt['z']
        return rates

class ObsLorenz63(pypfilt.obs.Univariate):

    def distribution(self, ctx, snapshot):
        expect = snapshot.state_vec[self.unit]
        return scipy.stats.norm(loc=expect, scale=1.5)

In [3]:
def check_lorenz63_instances():
    """Ensure that the example Lorenz-63 file defines a single scenario."""
    scenario_file = 'lorenz63_simulate.toml'
    instances = list(pypfilt.load_instances(scenario_file))
    assert len(instances) == 1

def simulate_lorenz63_observations():
    scenario_file = 'lorenz63_simulate.toml'
    instances = list(pypfilt.load_instances(scenario_file))
    instance = instances[0]

    # Simulate observations for x(t), y(t), and z(t).
    obs_tables = pypfilt.simulate_from_model(instance)

    # Save the observations to plain-text files.
    for obs_unit, obs_table in obs_tables.items():
        out_file = f'lorenz63-{obs_unit}.ssv'
        pypfilt.io.write_table(out_file, obs_table)

    return obs_tables

check_lorenz63_instances()

In [6]:
def run_lorenz63_forecast(filename=None):
    scenario_file = 'lorenz63_forecast.toml'
    instances = list(pypfilt.load_instances(scenario_file))
    instance = instances[0]

    # Run a forecast from t = 20.
    forecast_time = 20
    context = instance.build_context()
    return pypfilt.forecast(context, [forecast_time], filename=filename)

run_lorenz63_forecast()

Results(estimation=Result(settings={'components': {'model': 'pypfilt.examples.lorenz.Lorenz63', 'time': 'pypfilt.Scalar', 'sampler': 'pypfilt.sampler.LatinHypercube', 'summary': 'pypfilt.summary.HDF5'}, 'time': {'start': 0.0, 'until': 25.0, 'steps_per_unit': 10, 'summaries_per_unit': 10, 'dt': 0.1, 'epoch': 0.0, 'sim_start': 20.0, 'sim_until': 25.0}, 'prior': {'sigma': {'name': 'constant', 'args': {'value': 10}}, 'rho': {'name': 'constant', 'args': {'value': 28}}, 'beta': {'name': 'constant', 'args': {'value': 2.66667}}, 'x': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}, 'y': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}, 'z': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}}, 'observations': {'x': {'model': 'pypfilt.examples.lorenz.ObsLorenz63', 'file': 'lorenz63-x.ssv', 'parameters': {}}, 'y': {'model': 'pypfilt.examples.lorenz.ObsLorenz63', 'file': 'lorenz63-y.ssv', 'parameters': {}}, 'z': {'model': 'pypfilt.examples.lorenz.ObsLorenz63', 'file': 'lorenz63-z.ssv

In [7]:
class Lorenz63(pypfilt.OdeModel):
    def field_types(self, ctx):
        r"""
        Define the state vector :math:`[\sigma, \rho, \beta, x, y, z]^T`.
        """
        return [
            ('sigma', float),
            ('rho', float),
            ('beta', float),
            ('x', float),
            ('y', float),
            ('z', float),
        ]

    def d_dt(self, time, xt, ctx, is_forecast):
        """
        The right-hand side of the ODE system.

        :param time: The current time.
        :param xt: The particle state vectors.
        :param ctx: The simulation context.
        :param is_forecast: True if this is a forecasting simulation.
        """
        rates = np.zeros(xt.shape, xt.dtype)
        rates['x'] = xt['sigma'] * (xt['y'] - xt['x'])
        rates['y'] = xt['x'] * (xt['rho'] - xt['z']) - xt['y']
        rates['z'] = xt['x'] * xt['y'] - xt['beta'] * xt['z']
        return rates

    def can_smooth(self):
        """Indicate which state vector fields can be smoothed."""
        return {'sigma', 'rho', 'beta', 'x', 'y', 'z'}

def run_lorenz63_forecast_regularised(filename=None):
    scenario_file = 'lorenz63_forecast_regularised.toml'
    instances = list(pypfilt.load_instances(scenario_file))
    instance = instances[0]

    # Run a forecast from t = 20.
    forecast_time = 20
    context = instance.build_context()
    return pypfilt.forecast(context, [forecast_time], filename=filename)

run_lorenz63_forecast_regularised()

Results(estimation=Result(settings={'components': {'model': 'pypfilt.examples.lorenz.Lorenz63', 'time': 'pypfilt.Scalar', 'sampler': 'pypfilt.sampler.LatinHypercube', 'summary': 'pypfilt.summary.HDF5'}, 'time': {'start': 0.0, 'until': 25.0, 'steps_per_unit': 10, 'summaries_per_unit': 10, 'dt': 0.1, 'epoch': 0.0, 'sim_start': 20.0, 'sim_until': 25.0}, 'prior': {'sigma': {'name': 'constant', 'args': {'value': 10}}, 'rho': {'name': 'constant', 'args': {'value': 28}}, 'beta': {'name': 'constant', 'args': {'value': 2.66667}}, 'x': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}, 'y': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}, 'z': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}}, 'summary': {'tables': {'forecasts': {'component': 'pypfilt.summary.PredictiveCIs', 'credible_intervals': [50, 60, 70, 80, 90, 95]}, 'sim_z': {'component': 'pypfilt.summary.SimulatedObs', 'observation_unit': 'z'}}, 'save_history': False, 'only_forecasts': False, 'metadata': {'packages': []}},

In [12]:
import pypfilt.crps

def score_lorenz63_forecasts():
    """Calculate CRPS values for the simulated `z(t)` observations."""
    # Load the true observations that occur after the forecasting time.
    columns = [('time', float), ('value', float)]
    z_true = pypfilt.io.read_table('lorenz63-z.ssv', columns)
    z_true = z_true[z_true['time'] > 20]

    # Run the original forecasts.
    fs_file = 'lorenz63_forecast.hdf5'
    fs = run_lorenz63_forecast(filename=fs_file)

    # Run the forecasts with regularisation.
    reg_file = 'lorenz63_regularised.hdf5'
    fs_reg = run_lorenz63_forecast_regularised(filename=reg_file)

    # Extract the simulated z(t) observations for each forecast.
    time = pypfilt.Scalar()
    z_table = '/tables/sim_z'
    z_fs = pypfilt.io.load_summary_table(time, fs_file, z_table)
    z_reg = pypfilt.io.load_summary_table(time, reg_file, z_table)

    # Calculate CRPS values for each forecast.
    crps_fs = pypfilt.crps.simulated_obs_crps(z_true, z_fs)
    crps_reg = pypfilt.crps.simulated_obs_crps(z_true, z_reg)

    # Check that regularisation improved the forecast performance.
    assert np.mean(crps_reg['score']) < np.mean(crps_fs['score'])

    # Compare the CRPS values for each forecast.
    return (fs, fs_reg)

score_lorenz63_forecasts()

(Results(estimation=Result(settings={'components': {'model': 'pypfilt.examples.lorenz.Lorenz63', 'time': 'pypfilt.Scalar', 'sampler': 'pypfilt.sampler.LatinHypercube', 'summary': 'pypfilt.summary.HDF5'}, 'time': {'start': 0.0, 'until': 25.0, 'steps_per_unit': 10, 'summaries_per_unit': 10, 'dt': 0.1, 'epoch': 0.0, 'sim_start': 20.0, 'sim_until': 25.0}, 'prior': {'sigma': {'name': 'constant', 'args': {'value': 10}}, 'rho': {'name': 'constant', 'args': {'value': 28}}, 'beta': {'name': 'constant', 'args': {'value': 2.66667}}, 'x': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}, 'y': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}, 'z': {'name': 'uniform', 'args': {'loc': -5, 'scale': 10}}}, 'observations': {'x': {'model': 'pypfilt.examples.lorenz.ObsLorenz63', 'file': 'lorenz63-x.ssv', 'parameters': {}}, 'y': {'model': 'pypfilt.examples.lorenz.ObsLorenz63', 'file': 'lorenz63-y.ssv', 'parameters': {}}, 'z': {'model': 'pypfilt.examples.lorenz.ObsLorenz63', 'file': 'lorenz63-z.ss